In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
import wandb
project_name = "object_detection"
wandb.init(project=project_name, reinit=True)

wandb.run.name = 'augmentation.ipynb'
# generted run ID로 하고 싶다면 다음과 같이 쓴다.
# wandb.run.name = wandb.run.id
wandb.run.save()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jongeson (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/trash/detectors/detectors_r50_1x_trash.py')
PREFIX = '../../input/data/'


In [4]:
print(cfg.dump())

model = dict(
    type='CascadeRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True),
        output_img=True),
    neck=dict(
        type='RFP',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        rfp_steps=2,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=Tru

In [5]:
cfg.log_config.hooks[1].init_kwargs.project = project_name
cfg.log_config.hooks[1].init_kwargs.name = wandb.run.name
cfg.optimizer = dict(type = 'Adam', lr = 0.0001)
cfg.lr_config.step = [13, 17]
cfg.runner.max_epochs = 35
# cfg.runner.max_epochs = 4

In [6]:
##### albu_augmentation
albu_train_transforms = [
    dict(
        type='Resize',
        width=512,
        height=512,
        p=1.0),
    dict(
        type='Rotate',
        limit=10,
        p=0.5), 
    dict(
        type='RandomBrightnessContrast',
        p=1.0),
]
#####

In [9]:
# img_norm_cfg
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='Pad', size_divisor=32),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='DefaultFormatBundle'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels'],
        meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
                   'pad_shape', 'scale_factor')
    )
]

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)


cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)


# num_workers?
cfg.data.samples_per_gpu = 4

# seed
cfg.seed=2020

# ??
cfg.gpu_ids = [0]

# 새롭게 만든 모델을 저장할 장소
cfg.work_dir = './work_dirs/detectors_r50_1x_trash'

# class 개수 11개로 수정
cfg.model.roi_head.bbox_head[0].num_classes = 11
cfg.model.roi_head.bbox_head[1].num_classes = 11
cfg.model.roi_head.bbox_head[2].num_classes = 11

# ??
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [10]:
model = build_detector(cfg.model)

2021-05-18 07:29:52,882 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-18 07:29:52,885 - mmdet - INFO - Use load_from_torchvision loader
2021-05-18 07:29:53,174 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

In [11]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.30s)
creating index...
index created!


In [9]:

# # cfg.checkpoint_config

# wandb.login()
# cfg.log_config.hooks = [
#     dict(type='TextLoggerHook'),
#     dict(
#         type='WandbLoggerHook',
#         init_kwargs=dict(
#             project='p32',
#             name=save_file_name
#         ),
#         with_step=False # 아우 자꾸 저장 해대 ㅡㅡ
#     )
# ]


In [10]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-18 02:40:13,080 - mmdet - INFO - Start running, host: root@a868eb526110, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/detectors_r50_1x_trash
2021-05-18 02:40:13,119 - mmdet - INFO - workflow: [('train', 1)], max: 20 epochs
Done (t=1.00s)
creating index...
index created!
/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:113: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')


RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same